In [1]:
from unittest.mock import DEFAULT
from selenium import webdriver
import csv
import datetime
import schedule
import time
import sys
import configparser
import pandas as pd

#? ---configparserの宣言 iniファイルの読み込み---
#* configのディレクトリを記入する .read('●●●●●●●●' , encording='utf-8')
config_ini = configparser.ConfigParser()
config_ini.read('baba_condition_config.ini', encoding='utf-8')

#? -----定数定義-----
#* config.ini内で指定している
# webdriverのパス
driver = webdriver.Chrome(executable_path= config_ini.get('DEFAULT', 'DRIVER'))
# 出力履歴用csvを保存するパス
baba_condition_path = config_ini.get('DEFAULT', 'OUTPUT_FOLDER_PATH')
# 馬場状態を保存するDBcsvのパス/ファイル名
baba_condition_DB_filepath = config_ini.get('DEFAULT', 'DB_FILE_PATH')

#? -----関数定義-----
#* 対象のxpathの要素を探してテキストにする関数を定義する
# nameは変数名、xpathは取得したいパス
def to_text(name,xpath):
    name = driver.find_element_by_xpath(xpath)
    name = name.text
    return name

# URLを読み込んで馬場情報を取得する関数
def get_baba_condition(url):
    driver.get(url)

    # 明示的な待機
    driver.implicitly_wait(20)

    # 空のリストを作成しておく
    list = []

    # 取得時刻
    now = datetime.datetime.now()
    date = now.strftime("%Y%m%d_%H%M%S")
    list.append(date)

    # 競馬場名
    # 第1回中山競馬第2日前日
    # 中山だけを抜き出す
    name = driver.find_element_by_xpath('//*[@id="condition"]/div[1]/div/div/h2')
    name = name.text
    index1 = name.find('回')
    index2 = name.find('競')
    name = name[index1+1:index2]
    list.append(name)

    # 開催日 E.g)（2022年1月7日（金曜））
    name = driver.find_element_by_xpath('//*[@id="condition"]/div[1]/div/div/h2/span')
    name = name.text
    index1 = name.find('（')
    index2 = name.find('日')
    name = name[index1+1:index2+1]
    list.append(name)

    # # 馬場状態の日時
    # list.append(to_text('baba_condition_date', '//*[@id="course_condition"]/div[1]/div/div[1]/h3/span[1]'))
    # # 天候
    # list.append(to_text('weather','//*[@id="course_condition"]/div[2]/div[1]/div/div/div[2]/strong'))
    # # 馬場状態の芝
    # list.append(to_text('baba_condition_grass', '//*[@id="course_condition"]/div[2]/div[2]/ul/li/div/div[2]/p'))
    # # 馬場状態のダート
    # list.append(to_text('baba_condition_dirt', '//*[@id="course_condition"]/div[2]/div[3]/ul/li/div/div[2]/p'))

    # クッション値の測定時刻
    list.append(to_text('cushion_measurement_time', '//*[@id="cushion_list"]/option[1]'))
    # クッション値
    list.append(to_text('cushion_value', '//*[@id="cushion_num"]/p/strong'))
    # 含水率の測定時刻
    list.append(to_text('wc_measurement_time', '//*[@id="moist_list"]/option[1]'))
    # 含水率/芝-ゴール前
    list.append(to_text('wc_grass_goal', '//*[@id="turf_line"]/td[1]'))
    # 含水率/芝-4コーナー
    list.append(to_text('wc_grass_4corner', '//*[@id="turf_line"]/td[2]'))
    # 含水率/ダート-ゴール前
    list.append(to_text('wc_dirt_goal', '//*[@id="dirt_line"]/td[1]'))
    # 含水率/ダート-4コーナー
    list.append(to_text('wc_dirt_4corner', '//*[@id="dirt_line"]/td[2]'))

    print(list)

    return list

#* 出力履歴用csvを保存する
def output_tmp_data():
    #* 新規にcsvを作成する
    # 取得年月日時分秒
    now = datetime.datetime.now()
    date = now.strftime("_%Y%m%d_%H%M%S")

    filepath = baba_condition_path + "baba_condition" + date + ".csv"

    with open(filepath, 'w', newline="") as csv_file:
        fieldnames = ['取得時刻', '競馬場名','開催日','クッション値_測定時刻','クッション値','含水率_測定時刻','含水率_芝_ゴール前','含水率_芝4コーナー','含水率_ダート_ゴール前','含水率_ダート_4コーナー']
        writer = csv.writer(csv_file)
        writer.writerow(fieldnames)

    # データを取得
    with open(filepath, 'a', newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(get_baba_condition(config_ini.get('DEFAULT', 'URL1')))
        writer.writerow(get_baba_condition(config_ini.get('DEFAULT', 'URL2')))
        writer.writerow(get_baba_condition(config_ini.get('DEFAULT', 'URL3')))

    # webdriverの停止
    driver.quit()

    return filepath

#* 取得したtmp_dataをDBに入れる
def input_to_DB(file):
    tmp = pd.read_csv(file,encoding="shift-jis")

    # データフレーム型に格納
    df2 = pd.DataFrame(tmp)

    # DBをデータフレーム型に格納
    df1 = pd.DataFrame(pd.read_csv(baba_condition_DB_filepath))

    # DBと取得したcsvを結合する
    df = pd.concat([df1,df2])

    #重複を削除
    sindf = df.drop_duplicates(subset=['競馬場名', '開催日', 'クッション値_測定時刻'])
    sindf

    # CSVで保存
    sindf.to_csv(baba_condition_DB_filepath, index=False, encoding='utf-8-sig')

# プログラム実行して、8:45分に実行する
# 即時実行するならこのまま
# schedule.every().day.at("08:45").do(task)

#* プログラムの実行
input_to_DB(output_tmp_data())

#* プログラムの停止
sys.exit()

while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\winwin01\AppData\Local\Temp/ipykernel_21276/3571798806.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= config_ini.get('DEFAULT', 'DRIVER'))
C:\Users\winwin01\AppData\Local\Temp/ipykernel_21276/3571798806.py:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  name = driver.find_element_by_xpath('//*[@id="condition"]/div[1]/div/div/h2')
C:\Users\winwin01\AppData\Local\Temp/ipykernel_21276/3571798806.py:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  name = driver.find_element_by_xpath('//*[@id="condition"]/div[1]/div/div/h2/span')
C:\Users\winwin01\AppData\Local\Temp/ipykernel_21276/3571798806.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  name = driver.find_element_by_xpath(xpath)


['20220320_104818', '中山', '2022年3月20日', '3月20日（日曜）7時00分', '8.7', '3月20日（日曜）5時00分', '16.1%', '14.9%', '15.8%', '14.2%']
['20220320_104818', '阪神', '2022年3月20日', '3月20日（日曜）7時30分', '9.6', '3月20日（日曜）5時00分', '10.0%', '13.5%', '13.0%', '16.2%']
['20220320_104819', '中京', '2022年3月20日', '3月20日（日曜）6時30分', '8.2', '3月20日（日曜）6時30分', '14.2%', '14.0%', '9.6%', '10.6%']


SystemExit: 

d:\Users\winwin01\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
